- extend integration to manifolds in such a way that the integral is independent of the coordinate system used to compute it. 

In [1]:
(require '[clojupyter.misc.helper :as helper])
(run! helper/add-dependencies '[[sicmutils "0.18.0"]])

(ns chp5
    (:refer-clojure :exclude [partial zero? + - * / ref compare])
    (:require [sicmutils.env :refer :all]
              [sicmutils.value :as v]
              [sicmutils.calculus.manifold :refer [Rn, R2, make-patch, make-manifold
                                                   coordinate-system-at]]
              [sicmutils.mechanics.rotation :refer [Euler->M, rotate-x-matrix
                                                    rotate-y-matrix, rotate-z-matrix]]))
(def render (comp ->infix simplify))

Execution error (IllegalStateException) at chp5/eval4102$loading (REPL:4).
Rn already refers to: #'sicmutils.env/Rn in namespace: chp5


class java.lang.IllegalStateException: 

In [5]:
(def R3-rect-point (typical-point R3-rect))
;; (def R3-rect-point ((point R3-rect) (up 'x0 'y0 'z0)))

#'chp5/R3-rect-point

In [6]:
(let-coordinates
 [[x y z] R3-rect]
 (def u (+ (* 'u↑0 d:dx) (* 'u↑1 d:dy))) ;; constant vector field
 (def v (+ (* 'v↑0 d:dx) (* 'v↑1 d:dy)))
 (((wedge dx dy) u v) R3-rect-point))

(+ (* u↑0 v↑1) (* -1 v↑0 u↑1))

In [7]:
(let-coordinates
 [[r theta z] R3-cyl]
 (def a (+ (* 'a↑0 d:dr) (* 'a↑1 d:dtheta)))
 (def b (+ (* 'b↑0 d:dr) (* 'b↑1 d:dtheta)))
 (((wedge dr dtheta) a b) ((point R3-cyl) (up 'r0 'theta0 'z0))))

(+ (* a↑0 b↑1) (* -1 b↑0 a↑1))

- the two-form field acting on two vector fields $(dx \wedge dy) (u, v)(m)$ returns the area of the parallelogram in the $(x, y)$ coordinate plane.
- the area is in the coordinate plane. It is not the area on the manifold.

In [7]:
(let-coordinates
 [[x y z] R3-rect]
 (def u (+ (* 'u↑0 d:dx) (* 'u↑1 d:dy) (* 'u↑2 d:dz))) ;; constant vector field
 (def v (+ (* 'v↑0 d:dx) (* 'v↑1 d:dy) (* 'v↑2 d:dz)))
 (((wedge dx dz) u v) R3-rect-point))

(+ (* u↑0 v↑2) (* -1 v↑0 u↑2))

In [8]:
(let-coordinates
 [[x y z] R3-rect]
 (def u (+ (* 'u↑0 d:dx) (* 'u↑1 d:dy) (* 'u↑2 d:dz))) ;; constant vector field
 (def v (+ (* 'v↑0 d:dx) (* 'v↑1 d:dy) (* 'v↑2 d:dz)))
 (def w (+ (* 'w↑0 d:dx) (* 'w↑1 d:dy) (* 'w↑2 d:dz)))
 (def R3-rect-volume (((wedge dx dy dz) u v w) R3-rect-point)))

#'chp5/R3-rect-volume

In [9]:
R3-rect-volume

(* 1/2 (+ (* u↑0 (+ (* v↑1 w↑2) (* -1 w↑1 v↑2))) (* -1 u↑0 (+ (* w↑1 v↑2) (* -1 v↑1 w↑2))) (* w↑0 (+ (* u↑1 v↑2) (* -1 v↑1 u↑2))) (* -1 w↑0 (+ (* v↑1 u↑2) (* -1 u↑1 v↑2))) (* v↑0 (+ (* w↑1 u↑2) (* -1 u↑1 w↑2))) (* -1 v↑0 (+ (* u↑1 w↑2) (* -1 w↑1 u↑2)))))

In [10]:
(render
(- R3-rect-volume
   (determinant
    (matrix-by-rows '[u↑0 u↑1 u↑2]
                    '[v↑0 v↑1 v↑2]
                    '[w↑0 w↑1 w↑2]))))

"0"

### Ex 5.1 
Pick a coordinate system and use the computer to verify that 

__a.__ the wedge product is associative for forms in your coordinate system;

__b.__ formula (5.17) is true in your coordinate system

__Solution:__
__a.__ the following block shows
$$ (\mathsf{dx} \wedge (\mathsf{dy} \wedge \mathsf{dz}))(\mathsf{u}, \mathsf{v}, \mathsf{w})(\mathsf{m}) - ((\mathsf{dx} \wedge \mathsf{dy}) \wedge \mathsf{dz})(\mathsf{u}, \mathsf{v}, \mathsf{w})(\mathsf{m}) = 0 $$
in R3 coordinate

In [11]:
;; verify associativeness in R3-rect
(let-coordinates
 [[x y z] R3-rect]
 (def u (+ (* 'u↑0 d:dx) (* 'u↑1 d:dy) (* 'u↑2 d:dz))) ;; constant vector field
 (def v (+ (* 'v↑0 d:dx) (* 'v↑1 d:dy) (* 'v↑2 d:dz)))
 (def w (+ (* 'w↑0 d:dx) (* 'w↑1 d:dy) (* 'w↑2 d:dz)))
 (render
  (- (((wedge dx (wedge dy dz)) u v w) R3-rect-point)
     (((wedge (wedge dx dy) dz) u v w) R3-rect-point))))

"0"

__Solution: b.__

In [13]:
(let-coordinates
 [[x y z] R3-rect]
 (render
  (((wedge dx dy dz) d:dx d:dy d:dz) R3-rect-point)))

"1"

## 5.2 Exterior Derivative

The following block shows that for $\mathsf{\theta} = \mathsf{a dx} + \mathsf{b dy} + \mathsf{c dz}$,
$$
\mathsf{d \theta} = \mathsf{da} \wedge \mathsf{dx}
                   +\mathsf{db} \wedge \mathsf{dy}
                   +\mathsf{dc} \wedge \mathsf{dz}
$$

In [14]:
(def a (literal-manifold-function 'alpha R3-rect))
(def b (literal-manifold-function 'beta R3-rect))
(def c (literal-manifold-function 'gamma R3-rect))
(def X (literal-vector-field 'X-rect R3-rect))
(def Y (literal-vector-field 'Y-rect R3-rect))


(let-coordinates
 [[x y z] R3-rect]
 (def theta (+ (* a dx) (* b dy) (* c dz))) ;; general one-form field
 (render 
  (((- (d theta) 
       (+ (wedge (d a) dx)
          (wedge (d b) dy)
          (wedge (d c) dz)))
    X Y)
   R3-rect-point)))

"0"

The following block shows that for $\mathsf{\omega} = \mathsf{a} \mathsf{dy} \wedge \mathsf{dz} + 
                                                      \mathsf{b} \mathsf{dz} \wedge \mathsf{dx} + 
                                                      \mathsf{c} \mathsf{dx} \wedge \mathsf{dy}$,
$$
\mathsf{d\omega} = \mathsf{da} \wedge \mathsf{dy} \wedge \mathsf{dz} + 
                  \mathsf{db} \wedge \mathsf{dz} \wedge \mathsf{dx} + 
                  \mathsf{dc} \wedge \mathsf{dx} \wedge \mathsf{dy}
$$

In [15]:
(def Z (literal-vector-field 'Z-rect R3-rect))

(let-coordinates
 [[x y z] R3-rect]
 (def omega (+ (* a (wedge dy dz))
               (* b (wedge dz dx))
               (* c (wedge dx dy)))) ;; a general two form field
 (render
  (((- (d omega)
       (+ (wedge (d a) dy dz)
          (wedge (d b) dz dx)
          (wedge (d c) dx dy)))
    X Y X)
   R3-rect-point)))

"0"

- A form field $\mathsf{\omega}$ that is the exterior derivative of aother form field $\mathsf{d\theta}$ is called _exact_
- A form field whose exterior derivative is zero is call _closed_
- Every exact form field is a closed form field $\mathsf{d^2 \omega} = 0$, this is equivalent to that partial derivatives with respect to different variables commute

In [17]:
(render (((d (d theta)) X Y Z) R3-rect-point))

"0"

## Vector Integral Theorems
Stoke's Theorem
$$
\int_{\partial \mathsf{M}} \mathsf{\omega} = \int_{\mathsf{M}} \mathsf{d\omega}
$$
Green's Theorem, for an arbitrary compact set $M \subset \mathbb{R}^2$
$$
\int_{\partial \mathsf{M}} ((\alpha \circ \chi) \mathsf{dx} + (\beta \circ \chi) \mathsf{dy})
= \int_{\mathsf{M}} ((\partial_0 \beta - \partial_1 \alpha) \circ \chi) \mathsf{dx} \wedge \mathsf{dy}
$$

In [35]:
(def R2->R '(-> (UP Real Real) Real))
(def R2-rect-point ((point R2-rect) (up 'x0 'y0)))
(def v (literal-vector-field 'v-rect R2-rect))
(def w (literal-vector-field 'w-rect R2-rect))
(def α (literal-function 'alpha R2->R))
(def β (literal-function 'beta R2->R))

(def R2-rect-basis (coordinate-system->basis R2-rect))
(def dx (nth (basis->oneform-basis R2-rect-basis) 0))
(def dy (nth (basis->oneform-basis R2-rect-basis) 1))
(render
(((- (d (+ (* (compose α (chart R2-rect)) dx)
           (* (compose β (chart R2-rect)) dy)))
     (* (compose (- ((partial 0) β)
                    ((partial 1) α))
                 (chart R2-rect))
        (wedge dx dy)))
  v w)
 R2-rect-point))

"0"

- Divergence Theorem: for an arbitrary compact set $M \subset \mathbb{R}^3$ and a vector field $\mathsf{w}$
$$
\int_{\partial M} \mathsf{w \cdot n} dA = \int_M \textrm{div}(\mathsf{w}) dV
$$
- the integrands should be related by an exterior derivative, if this is an instance of Stokes's Theorem
- for orthonormal rectangular coordinates in $\mathbb{R}^3$, we can interpret the integrands in terms of forms. Let the vector field be
$
\mathsf{w} = \mathsf{a} \frac{\partial}{\partial \mathsf{x}} + 
\mathsf{b} \frac{\partial}{\partial \mathsf{y}} +
\mathsf{c} \frac{\partial}{\partial \mathsf{z}}
$
- The rate of leakage of stuff through each element of the boundary is $\mathsf{w} \cdot \mathsf{n} dA = \mathsf{a dy} \wedge \mathsf{dz} + \mathsf{b dz} \wedge \mathsf{dx} + \mathsf{c dx} \wedge \mathsf{dy}$
- The rate of production of stuff in each element of volume is $\textrm{div}(\mathsf{w}) dV = \big(\frac{\partial}{\partial \mathsf{x}} \mathsf{a} + 
\frac{\partial}{\partial \mathsf{y}} \mathsf{b} + 
\frac{\partial}{\partial \mathsf{z}} \mathsf{c} \big)
\mathsf{dx} \wedge \mathsf{dy} \wedge \mathsf{dz}$

In [18]:
(def a (literal-manifold-function 'a-rect R3-rect))
(def b (literal-manifold-function 'b-rect R3-rect))
(def c (literal-manifold-function 'c-rect R3-rect))

(let-coordinates
 [[x y z] R3-rect]
 (def flux-through-boundary-element
     (+ (* a (wedge dy dz))
        (* b (wedge dz dx))
        (* c (wedge dx dy))))
 (def production-in-volumn-element
     (* (+ (d:dx a) (d:dy b) (d:dz c))
        (wedge dx dy dz))))

#'chp5/production-in-volumn-element

In [20]:
(render
(((- production-in-volumn-element
     (d flux-through-boundary-element))
  X Y Z)
 R3-rect-point))

"0"